# Copyright 2022 Cognite AS

## Import the Libraries and Modules

In [ ]:
import sys
from pathlib import Path

utils = str(Path("../utils").resolve())
if utils not in sys.path:
    sys.path.append(utils)

from cognite_auth import interactive_client

## Create the Cognite Client

In [ ]:
c = interactive_client()

## Create various resource types

Create timeseries, datasets, labels, events etc.

**Code Pattern**

`client.<cdf_resource_type>.create()`

Dataset

In [ ]:
from cognite.client.data_classes import DataSet

In [ ]:
c.data_sets.list()

Add your prefix to make sure that the data we create are unique
You can for example use your name as a prefix

In [ ]:
prefix = "my_name"

In [ ]:
ds = c.data_sets.create(DataSet(name=prefix+"_example_dataset",external_id=prefix))
ds

When we create the other data objects, we will make sure they all goes into your data set

In [ ]:
ds_id = ds.id

Labels

In [ ]:
from cognite.client.data_classes import LabelDefinition
labels = [
    LabelDefinition(
        external_id="ROTATING_EQUIPMENT_" + prefix,
        name="Rotating equipment",
        data_set_id=ds_id,
    ),
    LabelDefinition(
        external_id="PUMP_" + prefix,
        name="pump",
        data_set_id=ds_id,
    )
]
res = c.labels.create(labels)

In [ ]:
c.labels.list()

Assets

In [ ]:
from cognite.client.data_classes import Asset
assets = [Asset(name="asset1",external_id="asset1_"+prefix,data_set_id=ds_id), 
            Asset(name="asset2",external_id="asset2_"+prefix,data_set_id=ds_id)]
res = c.assets.create(assets)

In [ ]:
c.assets.list(data_set_ids=ds_id)

Timeseries

In [ ]:
from cognite.client.data_classes import TimeSeries
ts = c.time_series.create(TimeSeries(name="timeseries_"+prefix,external_id = "timeseries_"+prefix,data_set_id=ds_id ))

In [ ]:
ts

Sequences

In [ ]:
from cognite.client.data_classes import Sequence
column_def = [{"valueType":"STRING","externalId":"user","description":"some description"},
             {"valueType":"DOUBLE","externalId":"amount"}]
seq = c.sequences.create(Sequence(external_id=prefix+"_test_sequence",name=prefix+" test sequence", columns=column_def,data_set_id=ds_id ))

Files metadata

In [ ]:
from cognite.client.data_classes import FileMetadata
file_metadata = FileMetadata(name=prefix+" file",external_id=prefix+"_file",data_set_id=ds_id)
res = c.files.create(file_metadata)

In [ ]:
res

Relationships

In [ ]:
from cognite.client.data_classes import Relationship
flowrel1 = Relationship(external_id=prefix+"_flow_12_test", source_external_id="asset1_"+prefix, source_type="asset", target_external_id="asset2_"+prefix, target_type="asset", confidence=0.1, data_set_id=ds_id)
flowrel2 = Relationship(external_id=prefix+"_flow_21_test", source_external_id="asset2_"+prefix, source_type="asset", target_external_id="asset1_"+prefix, target_type="asset", confidence=0.1, data_set_id=ds_id)
res = c.relationships.create([flowrel1,flowrel2])

In [ ]:
c.relationships.list()

Create Asset with Labels

In [ ]:
from cognite.client.data_classes import Label
asset = Asset(name=prefix+"_test_pump", labels=[Label(external_id="PUMP_"+prefix)],data_set_id=ds_id)
res = c.assets.create(asset)

In [ ]:
res

Create Asset Hierarchy

In [ ]:
assets_list = [Asset(external_id=prefix+"_root", name=prefix+" Root",data_set_id=ds_id),
          Asset(external_id=prefix+"_child1", parent_external_id=prefix+"_root", name="child1",data_set_id=ds_id), 
          Asset(external_id=prefix+"_child2", parent_external_id=prefix+"_root", name="child2",data_set_id=ds_id),
          Asset(external_id=prefix+"_child_of_child1", parent_external_id=prefix+"_child1", name="child_of_child1",data_set_id=ds_id)]
res = c.assets.create_hierarchy(assets_list)

In [ ]:
c.assets.retrieve_subtree(external_id=prefix+"_root")

## Update various resource types
**Code Pattern**

`client.<cdf_resource_type>.update()`

Full Update

In [ ]:
c.data_sets.list()

In [ ]:
data_set = c.data_sets.retrieve(id=ds_id)
data_set

In [ ]:
data_set.description = "This is a Example Dataset"
res = c.data_sets.update(data_set)

In [ ]:
res

Partial Update

In [ ]:
from cognite.client.data_classes import DataSetUpdate
my_update = DataSetUpdate(id=ds_id).description.set("New description with Partial Update")#.metadata.remove(["key"])
res = c.data_sets.update(my_update)

In [ ]:
res

In [ ]:
from cognite.client.data_classes import AssetUpdate
my_update = AssetUpdate(external_id="asset1_"+prefix).description.set("New description using AssetUpdate")#.metadata.add({"key": "value"})
res1 = c.assets.update(my_update)

In [ ]:
res1

Partial Update

In [ ]:
another_update = AssetUpdate(external_id="asset1_"+prefix).description.set(None)
res2 = c.assets.update(another_update)

In [ ]:
res2



---



In [ ]:
res = c.time_series.retrieve(external_id="timeseries_"+prefix)
res.description = "New description"
res = c.time_series.update(res)
res

In [ ]:
rel = c.relationships.retrieve(external_id=prefix+"_flow_12_test")
rel.confidence = 0.75
res = c.relationships.update(rel)
res

## Insert the data in CDF

**Code Pattern**
`client.<cdf_resource_type>.insert()`

### Insert the Datapoints/Rows

A datapoint consists of a value and a timestamp. We also need to tell which time series we want to write these datapoints into. 
We will use a time series that we created earlier and insert datapoints. 

In [ ]:
from datetime import datetime
# with datetime objects
datapoints = [(datetime(2020,1,1), 1000), (datetime(2020,1,2), 2000)]
c.time_series.data.insert(datapoints, external_id="timeseries_"+prefix)

We can use milliseconds when inserting datapoints as well

In [ ]:
datapoints = [(1577836800000, 800), (1580515200000, 500)]
c.time_series.data.insert(datapoints, external_id="timeseries_"+prefix)

Look at the datapoints that was created

In [ ]:
c.time_series.data.retrieve(external_id="timeseries_"+prefix,start=datetime(2020,1,1),end=datetime(2021,1,1))

In [ ]:
c.sequences.retrieve(external_id=prefix+"_test_sequence").columns

In [ ]:
data = [(1, ['user1',3.14]), (2, ['user2',2.72]) ]
c.sequences.data.insert(columns=["user","amount"], rows=data, external_id=prefix+"_test_sequence")

Look at the sequences that was created

In [ ]:
c.sequences.data.retrieve_dataframe(external_id=prefix+"_test_sequence",start=0,end=100)

### Insert the Dataframe

In [ ]:
import pandas as pd
from datetime import timedelta
import numpy as np

In [ ]:
ts_xid = "timeseries_"+prefix
x = pd.date_range(start="2023", freq="1d", periods=100)
y = np.random.normal(0, 1, 100)
df = pd.DataFrame({ts_xid: y}, index=x)
df

In [ ]:
c.time_series.data.insert_dataframe(df, external_id_headers=True)